# Investigation of how many observations in test set needed for good results

Still uses the same test sets, so loss values are comparable across models and with benchmark models

## First with most recent observations as test data

In [ ]:
%load_ext autoreload
%autoreload 3
from data_functions import get_dataset
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler as SRS
from train_utils import SubsetSampler as SS
from sklearn.model_selection import train_test_split as TTS
import torch, random, os, numpy as np
torch.use_deterministic_algorithms(True) # reproducibility

STATE = "SP"
WEEKS = False
TRIANGLE = True
PAST_UNITS = 40
MAX_DELAY = 40
BATCH_SIZE = 64
RANDOM_SPLIT = False
SEED = 1234
DEVICE = "mps"
DOW = True

dataset= get_dataset(weeks=WEEKS, triangle=TRIANGLE, past_units=PAST_UNITS, max_delay=MAX_DELAY, state=STATE, dow = DOW)
#n_obs_40pu = len(dataset) # 2922 total dates, -39-39 for past_units and max_delay ->2844
## Define train and test indices
if RANDOM_SPLIT:
    all_idcs = range(dataset_dow.__len__())
    train_idcs, test_idcs = TTS(all_idcs, test_size=0.25, shuffle=True, random_state=SEED)
    train_idcs, val_idcs = TTS(train_idcs, test_size=0.25, shuffle=True, random_state=SEED)
    #train_idcs, test_idcs = [*range(600), *range(950, dataset.__len__())], [*range(600, 950)]
    VAL_BATCH_SIZE, TEST_BATCH_SIZE = len(val_idcs), len(test_idcs)
else:
    if WEEKS:
        train_idcs, test_idcs = range(300), range(300, dataset.__len__())
        TEST_BATCH_SIZE = dataset.__len__() - 300
    else: 
        train_idcs, test_idcs = range(2133), range(2133, dataset.__len__()) # 2844 total obs - 711 test, still 25% even without random split dataset.__len__(), 2353
        train_idcs, val_idcs = TTS(train_idcs, test_size=0.25, shuffle=True, random_state=SEED)
        VAL_BATCH_SIZE, TEST_BATCH_SIZE = len(val_idcs), len(test_idcs)
        
## Define generator so sampling during training is deterministic and reproducible
g = torch.Generator()
g.manual_seed(SEED)
train_sampler, val_sampler, test_sampler = SRS(train_idcs, generator=g), SRS(val_idcs), SS(test_idcs)
train_loader, val_loader, test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler), DataLoader(dataset, batch_size=VAL_BATCH_SIZE, sampler=val_sampler, shuffle=False), DataLoader(dataset, batch_size=TEST_BATCH_SIZE, sampler=test_sampler_dow, shuffle=False)

## Function to reset the sampler so each training run uses same order of observations for reproducibility
## Possible to define s.t. returns train_loader, but bc in notebook, possible to define globally
def regen_data():
    g = torch.Generator()
    g.manual_seed(SEED)
    global train_loader_dow
    train_loader_dow = DataLoader(dataset_dow, batch_size=BATCH_SIZE, sampler=SRS(train_idcs, generator=g))

def set_seeds(SEED):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)
    random.seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

set_seeds(SEED)

def regen_data():
    g = torch.Generator()
    g.manual_seed(SEED)
    global train_loaders
    for i in range(FUTURE_OBS+1):
        train_loaders[i] = DataLoader(dataset_collection[i], batch_size=BATCH_SIZE, sampler=SRS(train_idcs, generator=g))
        
train_loaders, val_loaders, test_loaders = [], [], []
for i in range(FUTURE_OBS+1):
    train_loaders.append(DataLoader(dataset_collection[i], batch_size=BATCH_SIZE, sampler=SRS(train_idcs, generator=g)))
    val_loaders.append(DataLoader(dataset_collection[i], batch_size=len(val_idcs), sampler=SRS(val_idcs)))
    test_loaders.append(DataLoader(dataset_collection[i], batch_size=len(test_idcs), sampler=SRS(test_idcs)))


If the architecture is final, could just import from NowcastPNN

In [ ]:
import torch.nn as nn
import torch
from NegativeBinomial import NegBin as NB
set_seeds(SEED)
## For matrix-like (two-dimensional) input data
class NowcastPNNDOW(nn.Module):
    """ Still NowcastPNN, just this time processing the day of the week additionally to reporting triangle """
    def __init__(self, past_units = 40, max_delay = 40, hidden_units = [16, 8], conv_channels = [16, 1], embedding_dim = 10, load_embed = True):
        super().__init__()
        self.past_units = past_units
        self.max_delay = max_delay
        self.final_dim = past_units
        self.conv1 = nn.Conv1d(self.max_delay, conv_channels[0], kernel_size=7, padding="same")
        self.conv2 = nn.Conv1d(conv_channels[0], conv_channels[1], kernel_size=7, padding="same")
        self.fc1 = nn.Linear(self.past_units, self.past_units)#, nn.Linear(self.past_units, self.past_units)
        self.fc3, self.fc4 = nn.Linear(self.final_dim, hidden_units[0]), nn.Linear(hidden_units[0], hidden_units[1])#, nn.Linear(hidden_units[1], hidden_units[2])
        #self.fc5 = nn.Linear(hidden_units[1], hidden_units[2])
        self.fcnb = nn.Linear(hidden_units[-1], 2)
        self.const = 10000 # if not normalized, take constant out
        self.embedding_dim = embedding_dim
        if load_embed:
            self.embed = nn.Embedding.from_pretrained(torch.load(f"./weights/embedding_weights_{embedding_dim}").detach())
        else:
            self.embed = nn.Embedding(7, embedding_dim)
        #self.embed.weight.requires_grad_(False)
        #self.embed.weight = nn.Parameter(torch.randn((7, embedding_dim))), can use to initialize, doesn't help
        self.fc_embed1, self.fc_embed2 = nn.Linear(embedding_dim, 2*embedding_dim), nn.Linear(2*embedding_dim, past_units)
        #self.fc_embed1 = nn.Linear(embedding_dim, past_units)

        self.bnorm1, self.bnorm2 = nn.BatchNorm1d(num_features=self.max_delay), nn.BatchNorm1d(num_features=conv_channels[0])#, nn.BatchNorm1d(num_features=conv_channels[1])#, nn.BatchNorm1d(num_features=conv_channels[2])
        #self.bnorm3 = nn.BatchNorm1d(num_features=conv_channels[1])
        self.bnorm5, self.bnorm6  = nn.BatchNorm1d(num_features=self.final_dim), nn.BatchNorm1d(num_features=hidden_units[0])#, nn.BatchNorm1d(num_features=hidden_units[2])
        self.bnorm_embed = nn.BatchNorm1d(num_features=2*embedding_dim)
        #self.bnorm7 = nn.BatchNorm1d(num_features=hidden_units[1])
        self.bnorm_final = nn.BatchNorm1d(num_features=hidden_units[-1]) #hidden_units[1]/self.past_units for single model
        self.attn1 = nn.MultiheadAttention(embed_dim=self.max_delay, num_heads=1, batch_first=True)
        self.drop1, self.drop2 = nn.Dropout(0.1), nn.Dropout(0.1) 
        self.softplus = nn.Softplus()
        self.act = nn.SiLU()
    
    def save_embeddings(self):
        """ Allows the user to save the embeddings if trained with a different dimension
        to load later and allow for reproducible training runs. Usage: run model with load_embed = False,
        then use model.save_embeddings() after training and use the model with load_embed = True afterwards.
        """
        torch.save(self.embed.weight, f"./weights/embedding_weights_{self.embedding_dim}")
    
    def forward(self, rep_tri, dow): ## Feed forward function, takes input of shape [batch, past_units, max_delay]
        #x = x.permute(0, 2, 1) # [batch, past_units, max_delay] -> [batch, max_delay, past_units]
        x = rep_tri.float()

        ## Attention Block ##
        x_add = x.clone()
        x = self.attn1(x, x, x, need_weights = False)[0]
        x = self.act(self.fc1(x.permute(0,2,1)))
        x = x.permute(0,2,1) + x_add

        ## Convolutional Block ##
        x = x.permute(0, 2, 1) # [batch, past_units, max_delay] -> [batch, max_delay, past_units]
        x = self.act(self.conv1(self.bnorm1(x)))
        x = self.act(self.conv2(self.bnorm2(x)))
        #x = self.act(self.conv3(self.bnorm3(x)))
        #x = self.act(self.conv4(self.bnorm4(x)))
        x = torch.squeeze(x)
        #print(x)
        ## Addition of embedding of day of the week ##
        embedded = self.embed(dow)
        #print(embedded)
        x = x + self.act(self.fc_embed2(self.bnorm_embed(self.act(self.fc_embed1(embedded))))) # self.bnorm_embed1(embedded)
        ## Fully Connected Block ##
        x = self.drop1(x)
        x = self.act(self.fc3(self.bnorm5(x)))
        x = self.drop2(x)
        x = self.act(self.fc4(self.bnorm6(x)))
        #x = self.drop3(x)
        #x = self.act(self.fc5(self.bnorm7(x)))
        x = self.fcnb(self.bnorm_final(x))
        dist = NB(lbda = self.const*self.softplus(x[:, 0]), phi = (self.const**2)*self.softplus(x[:, 1])+1e-5)
        """
        x = self.fcpoi(self.bnorm_final(x))
        dist = Poi(rate=self.const*self.softplus(x)+1e-5)
        """
        return torch.distributions.Independent(dist, reinterpreted_batch_ndims=1)

In [ ]:
%load_ext autoreload
%autoreload 3
from NowcastPNN import NowcastPNNDOW
from train_utils import train, EarlyStopper

set_seeds(SEED)
n_training = [500, 1000, 1500, 2000] # originally 2133 as training + val
for n in n_training:
    print(f"##################### Number training data points: {n} #####################")
    ## Redefine training and validation indices, and define train and validation loaders
    regen_data() # reset samplers so each training run is reproducible
    early_stopper = EarlyStopper(patience=30, past_units=PAST_UNITS, max_delay=MAX_DELAY, weeks=WEEKS, random_split=RANDOM_SPLIT, dow = DOW, n_training = n)
    nowcast_pnn = NowcastPNNDOW(past_units=PAST_UNITS, max_delay=MAX_DELAY, embedding_dim=10)
    train(nowcast_pnn, num_epochs=200, train_loader=train_loaders[i], val_loader=val_loaders[i], early_stopper=early_stopper, loss_fct="nll", device = DEVICE, dow = DOW)
    ## Load best set of weights on test/validation set
    #nowcast_pnn.load_state_dict(torch.load(f"./weights/weights-{PAST_UNITS}-{MAX_DELAY}-{'week' if WEEKS else 'day'}-fut0{'-rec' if not RANDOM_SPLIT else ''}{'-dow' if DOW else ''}"))

In [ ]:
from metrics import evaluate_PIs, pnn_PIs
set_seeds(SEED) # Reproducible results

for i in range(FUTURE_OBS):
    print(f"##################### Number training data points: {n} #####################")
    nowcast_pnn.load_state_dict(torch.load(f"./weights/weights-{PAST_UNITS}-{MAX_DELAY}-{'week' if WEEKS else 'day'}-fut0{'-rec' if not RANDOM_SPLIT else ''}{'-dow' if DOW else ''}-{n}"))
    levels_pnn = pnn_PIs(nowcast_pnn, test_loaders[i], random_split = RANDOM_SPLIT, save=False, dow = DOW) # think about how save with name
    _ = evaluate_PIs(levels_pnn, test_loaders[i])